In [1]:
import requests
import pandas as pd

In [2]:
input_ror = '04vfs2w97'

# Sub-affiliations from RoR

In [30]:
def get_sub_rors(input_ror):
    sub_rors = [input_ror]
    url = f'https://api.ror.org/organizations/{input_ror}'
    res = requests.get(url).json()
    children = [c for c in res['relationships'] if c['type'] == 'Child']
    for c in children:
        children_ror = c['id'].split('/')[-1]
        sub_rors += get_sub_rors(children_ror)
    return list(set(sub_rors))
#sub_rorsget_sub_rors(input_ror)

In [42]:
def get_query(input_ror, level):
    url = f'https://api.ror.org/organizations/{input_ror}'
    res = requests.get(url).json()
    #sub_rors = [input_ror]
    query = []
    query_set = set([])
    if res.get('name'):
        query.append({'str': res['name'], 'type': f'ror_name_{level}', 'source': input_ror})
        query_set.add(res['name'])
    for a in res['aliases']:
        if a not in query_set:
            query.append({'str': a, 'type': f'ror_alias_{level}', 'source': input_ror})
            query_set.add(a)
    for label in res['labels']:
        a = label['label']
        if a not in query_set:
            query.append({'str': a, 'type': f'ror_label_{level}', 'source': input_ror})
            query_set.add(a)
    children = [c for c in res['relationships'] if c['type'] == 'Child']
    children_query = []
    #print(len(children))
    for c in children:
        children_ror = c['id'].split('/')[-1]
        #sub_rors.append(children_ror)
        print(children_ror, end=',')
        sub_query = get_query(children_ror, 'child')
        children_query += sub_query['query']
        #sub_rors += sub_query['rors']
    #sub_rors = list(set(sub_rors))
    #print(len(children)
    return {'query': query + children_query}#, 'rors': sub_rors}

In [43]:
x = get_query(input_ror, 'main')

02ynmc580,02fdf4056,05em8ne27,04ghtjd11,011nqat45,02cy9g557,025mhc977,01gbppd30,00avmbz91,01kwx4m12,04yc2e502,05k1smh27,04rvw8791,01petg633,03c4rpa03,05je79696,00gyb4719,055swm364,00p6sn362,01pfter31,03nknpw16,01xd0ys45,017je3b10,01nyrrx14,01e8etd40,02vnf0c38,022r5hc56,01d5v2p67,016vxbe23,0106d5c37,04n2vwk08,04xykxd72,01dg9a450,04mq2px33,02cyw3861,0371yc337,020azc393,02atkd403,04ev1zs55,03d28dw78,04x6jg419,05adajg67,05ft9wv42,019c1xz73,01evjjx71,053d03003,04f50n919,0495mzm65,018d4rr63,05yc49b49,02vc99v03,01hntn815,01sp9jb48,05kjgf606,04gmrh438,055yrx709,03hhwe765,03pz4q672,00xngy573,01e0n5141,00wbse129,052k52s41,00mny5z95,01fe8k477,00e8hwt93,05be9p317,01x6z5t49,01nrtdp55,0459fdx51,04yem5s35,01sgwka45,

In [44]:
df = pd.DataFrame(x['query'])
sub_rors = list(set(df.source.to_list()))
len(sub_rors)

72

# Affiliation names

In [46]:
def transform(x):
    x = x.replace(',', ' ').replace('  ', ' ').replace(' ', '+')
    return x

In [47]:
url = 'https://works-finder.staging.dataesr.ovh/#/?affiliations=' + "&affiliations=".join(df.str.apply(lambda x:transform(x)).to_list())
url += '&datasets=true&endYear=2023&startYear=2021'
print(url)

https://works-finder.staging.dataesr.ovh/#/?affiliations=Université+de+Lorraine&affiliations=University+of+Lorraine&affiliations=Animal+and+Functionality+of+Animal+Products&affiliations=Animal+et+Fonctionnalités+des+Produits+Animaux&affiliations=Archives+Henri-Poincaré+-+Philosophie+et+Recherches+sur+les+Sciences+et+les+Technologies&affiliations=Bureau+for+Economic+Theory+and+Applications&affiliations=Bureau+d'Economie+Théorique+et+Appliquée&affiliations=Centre+d'Investigation+Clinique+Pierre+Drouin&affiliations=CIC-Pierre+Drouin&affiliations=CIC-P+NANCY&affiliations=Centre+de+Recherches+Pétrographiques+et+Géochimiques&affiliations=Computer+Processing+and+Analysis+of+the+French+Language&affiliations=Analyse+et+Traitement+Informatique+de+la+Langue+Française&affiliations=Diagnosis+and+International+Adaptive+Imaging&affiliations=IADI+Lab&affiliations=Imagerie+Adaptative+Diagnostique+et+Interventionnelle&affiliations=Dynamique+des+Génomes+et+Adaptation+Microbienne&affiliations=DynAMic&affi

# Authors from OpenAlex

In [52]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [69]:
oa_results = []
oa_results_id = set([])

In [77]:
sub_rors_chunks = list(chunks(sub_rors, 50))
for chunk_ix, sub_rors_chunk in enumerate(sub_rors_chunks):
    sub_rors_str = '|'.join(sub_rors_chunk)
    print(len(sub_rors_chunk))
    for jx in range(0, 1000):
        print(f'chunk {chunk_ix} - {jx}', end=',')
        if jx == 0:
            cursor = '*'
        url_oa = f'https://api.openalex.org/authors?filter=affiliations.institution.ror:{sub_rors_str}&per-page=200&cursor={cursor}'
        res = requests.get(url_oa).json()
        if 'results' not in res:
            break
        for k in res['results']:
            if k['id'] not in oa_results_id:
                oa_results.append(k)
                oa_results_id.add(k['id'])
        try:
            cursor = res['meta']['next_cursor']
        except:
            break

50
chunk 0 - 0,chunk 0 - 1,chunk 0 - 2,chunk 0 - 3,chunk 0 - 4,chunk 0 - 5,chunk 0 - 6,chunk 0 - 7,chunk 0 - 8,chunk 0 - 9,chunk 0 - 10,chunk 0 - 11,chunk 0 - 12,chunk 0 - 13,chunk 0 - 14,chunk 0 - 15,chunk 0 - 16,chunk 0 - 17,chunk 0 - 18,chunk 0 - 19,chunk 0 - 20,chunk 0 - 21,chunk 0 - 22,chunk 0 - 23,chunk 0 - 24,chunk 0 - 25,chunk 0 - 26,chunk 0 - 27,chunk 0 - 28,chunk 0 - 29,chunk 0 - 30,chunk 0 - 31,chunk 0 - 32,chunk 0 - 33,chunk 0 - 34,chunk 0 - 35,chunk 0 - 36,chunk 0 - 37,chunk 0 - 38,chunk 0 - 39,chunk 0 - 40,chunk 0 - 41,chunk 0 - 42,chunk 0 - 43,chunk 0 - 44,chunk 0 - 45,chunk 0 - 46,chunk 0 - 47,chunk 0 - 48,chunk 0 - 49,chunk 0 - 50,chunk 0 - 51,22
chunk 1 - 0,chunk 1 - 1,chunk 1 - 2,chunk 1 - 3,chunk 1 - 4,chunk 1 - 5,chunk 1 - 6,chunk 1 - 7,chunk 1 - 8,chunk 1 - 9,chunk 1 - 10,chunk 1 - 11,chunk 1 - 12,chunk 1 - 13,chunk 1 - 14,chunk 1 - 15,chunk 1 - 16,chunk 1 - 17,chunk 1 - 18,chunk 1 - 19,chunk 1 - 20,chunk 1 - 21,chunk 1 - 22,chunk 1 - 23,chunk 1 - 24,chunk 1 - 25,

In [85]:
authors_to_search = []
for author in oa_results:
    to_add = False
    for aff in author['affiliations']:
        if isinstance(aff.get('institution')['ror'], str) and aff.get('institution')['ror'].split('/')[-1] in sub_rors:
            recent_years = [y for y in aff.get('years', []) if y > 2020]
            if len(recent_years)>=3:
                to_add = True
                break
    if to_add:
        authors_to_search.append(author)

In [88]:
len(authors_to_search)

1788

In [92]:
pd.DataFrame(authors_to_search)[['orcid', 'display_name']].head(25)

,orcid,display_name
0,https://orcid.org/0000-0002-4737-3715,Francis Martin
1,https://orcid.org/0000-0002-9749-7185,A. Pizzi
2,https://orcid.org/0000-0002-1001-6386,Guohua Hu
3,https://orcid.org/0000-0002-3743-9215,Éric Rondeau
4,https://orcid.org/0000-0002-9687-7952,Fabrice Laurent
5,https://orcid.org/0000-0002-1845-3935,Bruno Rossion
6,https://orcid.org/0000-0001-5236-9561,Amedeo Napoli
7,https://orcid.org/0000-0001-8067-7983,G. Amarenco
8,None,Didier Mainard
9,None,Patrick Netter
